# Technical indicators

### Import

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path 
import datatable as dt

from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from ta.momentum import AwesomeOscillatorIndicator

### Get data

In [2]:
filepath = Path('../../data/csv/crypto_market_data.csv')

In [3]:
df_crypto = dt.fread(filepath).to_pandas() 
df_crypto = df_crypto.T.set_index([0, 1]).T
df_crypto = df_crypto.set_index('')
df_crypto = df_crypto.astype(float)
df_crypto.index.names = ['date']

df_btc = df_crypto['BTC-USD']
df_btc.head(3)

1,Adj Close,Close,High,Low,Open,Volume
date,,,,,,
2020-04-07 00:00:00+00:00,7366.915527,7366.915527,7407.448730,7268.242188,7273.327637,0.000000e+00
2020-04-07 01:00:00+00:00,7355.732910,7355.732910,7383.069824,7350.611816,7361.706055,0.000000e+00
2020-04-07 02:00:00+00:00,7311.450195,7311.450195,7355.026367,7290.237793,7355.026367,1.082130e+09


### Get features

In [4]:
filepath = Path('../../features/csv/features_btc_ta.csv')

In [5]:
df = dt.fread(filepath).to_pandas() 
df = df.set_index('date')
df.head(3)

,BB_high,BB_low,BB_width,SMA_BBW,SMA_Vol,EMA_fast,EMA_slow,MACD,EMA_MACD,HH,LL
date,,,,,,,,,,,
2020-04-09 20:00:00+00:00,7329.607505,7251.915152,1.065627,-1.480833,2.109254e+08,7280.083462,7287.614550,7.531088,3.132471,7337.893066,7179.094238
2020-04-09 21:00:00+00:00,7327.833311,7250.361709,1.062842,-1.457221,2.290828e+08,7279.137478,7286.601218,7.463740,3.998724,7327.550781,7179.094238
2020-04-09 22:00:00+00:00,7325.640758,7249.332240,1.047117,-1.424009,4.009165e+07,7278.601154,7285.790118,7.188964,4.636772,7327.550781,7179.094238


In [6]:
df = df.join(df_btc, lsuffix='_ta', rsuffix='_btc')
df.head(3)

,BB_high,BB_low,BB_width,SMA_BBW,SMA_Vol,EMA_fast,EMA_slow,MACD,EMA_MACD,HH,LL,Adj Close,Close,High,Low,Open,Volume
date,,,,,,,,,,,,,,,,,
2020-04-09 20:00:00+00:00,7329.607505,7251.915152,1.065627,-1.480833,2.109254e+08,7280.083462,7287.614550,7.531088,3.132471,7337.893066,7179.094238,7254.513672,7254.513672,7273.875488,7235.329590,7271.894531,0.0
2020-04-09 21:00:00+00:00,7327.833311,7250.361709,1.062842,-1.457221,2.290828e+08,7279.137478,7286.601218,7.463740,3.998724,7327.550781,7179.094238,7273.934570,7273.934570,7276.812500,7259.898926,7261.062988,226967552.0
2020-04-09 22:00:00+00:00,7325.640758,7249.332240,1.047117,-1.424009,4.009165e+07,7278.601154,7285.790118,7.188964,4.636772,7327.550781,7179.094238,7275.651367,7275.651367,7284.396484,7268.491211,7275.088867,0.0


### Compute signals

In [7]:
df_signals = pd.DataFrame([])

#### 1. Conditions

In [8]:
def condition_BBW(df):
    return (df.SMA_BBW > 0)*1

def condition_Vol(df):
    return (df.SMA_Vol > 0)*1

In [9]:
df_signals['condition_BBW'] = condition_BBW(df)
df_signals['condition_Vol'] = condition_Vol(df)

#### 2. Strategies

In [10]:
def strat_BB(df, alpha):
    buy = (df['Close'] - df['BB_high'] > abs(df['Close'])*alpha)*1
    sell = (df['Close'] - df['BB_low'] < -abs(df['Close'])*alpha)*(-1)
    return buy + sell

def strat_MACD(df, alpha):
    buy = (df['MACD'] - df['EMA_MACD'] < -abs(df['MACD'])*alpha)*1
    sell = (df['MACD'] - df['EMA_MACD'] > abs(df['MACD'])*alpha)*(-1)
    return buy + sell

def strat_HH_LL(df):
    buy = (df['High'] - df['HH'] > 0)*1
    sell = (df['Low'] - df['LL'] < 0)*(-1)
    return buy + sell

In [11]:
df

,BB_high,BB_low,BB_width,SMA_BBW,SMA_Vol,EMA_fast,EMA_slow,MACD,EMA_MACD,HH,LL,Adj Close,Close,High,Low,Open,Volume
date,,,,,,,,,,,,,,,,,
2020-04-09 20:00:00+00:00,7329.607505,7251.915152,1.065627,-1.480833,2.109254e+08,7280.083462,7287.614550,7.531088,3.132471,7337.893066,7179.094238,7254.513672,7254.513672,7273.875488,7235.329590,7271.894531,0.0
2020-04-09 21:00:00+00:00,7327.833311,7250.361709,1.062842,-1.457221,2.290828e+08,7279.137478,7286.601218,7.463740,3.998724,7327.550781,7179.094238,7273.934570,7273.934570,7276.812500,7259.898926,7261.062988,226967552.0
2020-04-09 22:00:00+00:00,7325.640758,7249.332240,1.047117,-1.424009,4.009165e+07,7278.601154,7285.790118,7.188964,4.636772,7327.550781,7179.094238,7275.651367,7275.651367,7284.396484,7268.491211,7275.088867,0.0
2020-04-10 00:00:00+00:00,7326.467842,7249.426250,1.057110,-1.377465,-1.624236e+07,7281.885621,7286.839013,4.953391,4.700096,7327.550781,7179.094238,7299.950195,7299.950195,7302.565430,7277.189941,7302.565430,0.0
2020-04-10 01:00:00+00:00,7325.562422,7248.925419,1.051660,-1.320730,-2.388591e+07,7282.107470,7286.578911,4.471441,4.654365,7327.550781,7179.094238,7283.327637,7283.327637,7298.331543,7274.533691,7296.788574,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-06 20:00:00+00:00,45999.590200,43481.655894,5.627848,0.827987,2.093013e+08,44291.203420,44823.914594,532.711174,455.740835,45501.582031,43452.957031,43910.609375,43910.609375,43972.375000,43800.152344,43818.039062,80510976.0
2022-04-07 00:00:00+00:00,46040.274341,43246.709643,6.257496,1.076419,7.899120e+07,44130.976812,44707.308363,576.331550,479.858978,45501.582031,43208.417969,43249.730469,43249.730469,43541.945312,43208.417969,43211.718750,85118976.0
2022-04-07 00:52:00+00:00,46034.432037,43051.834370,6.695976,1.342379,1.220075e+08,43995.400452,44599.339630,603.939178,504.675018,45501.582031,43208.417969,43249.730469,43249.730469,43541.945312,43208.417969,43211.718750,85118976.0


In [12]:
df_signals['strat_BB'] = strat_BB(df, alpha=0.001)
df_signals['strat_MACD'] = strat_MACD(df, alpha=0.1)
df_signals['strat_HH_LL'] = strat_HH_LL(df)

In [14]:
df_signals.head(3)

,condition_BBW,condition_Vol,strat_BB,strat_MACD,strat_HH_LL
date,,,,,
2020-04-09 20:00:00+00:00,0,1,0,-1,0
2020-04-09 21:00:00+00:00,0,1,0,-1,0
2020-04-09 22:00:00+00:00,0,1,0,-1,0


### Save features

In [ ]:
filepath = Path('../csv/signals_btc_ta.csv')

In [ ]:
df_btc_ta.to_csv(filepath)